In [ ]:
# notebooks use their location as their working directory, so
# if we are in a subfolder, move to the main folder.  
# This however can safely be run multiple times
#setwd("M:/lecospec/lecospec")
if(!dir.exists("Functions/")){
    setwd("../")
    if(!dir.exists("Functions")){
        setwd("M:/lecospec/lecospec/")
    }
}
source("Functions/lecospectR.R", echo = FALSE)
library(class)
library(caret)
library(vegan)

## Load the Data

In [ ]:
# spectral library
base_path <- "./Output/C_001_SC3_Cleaned_SpectralLib.csv"
veg_index_path <- "./Data/D_002_SpecLib_Derivs.csv"
speclib <- read.csv(base_path)
veg_indices <- read.csv(veg_index_path)

In [ ]:
colnames(speclib)[12:2162]

In [ ]:
used_bands <- c(
    "X397.593_5nm",
    "X402.593_5nm",
    "X407.593_5nm",
    "X412.593_5nm",
    "X417.593_5nm",
    "X422.593_5nm",
    "X427.593_5nm",
    "X432.593_5nm",
    "X437.593_5nm",
    "X442.593_5nm",
    "X447.593_5nm",
    "X452.593_5nm",
    "X457.593_5nm",
    "X462.593_5nm",
    "X467.593_5nm",
    "X472.593_5nm",
    "X477.593_5nm",
    "X482.593_5nm",
    "X487.593_5nm",
    "X492.593_5nm",
    "X497.593_5nm",
    "X502.593_5nm",
    "X507.593_5nm",
    "X512.593_5nm",
    "X517.593_5nm",
    "X522.593_5nm",
    "X527.593_5nm",
    "X532.593_5nm",
    "X537.593_5nm",
    "X542.593_5nm",
    "X547.593_5nm",
    "X552.593_5nm",
    "X557.593_5nm",
    "X562.593_5nm",
    "X567.593_5nm",
    "X572.593_5nm",
    "X577.593_5nm",
    "X582.593_5nm",
    "X587.593_5nm",
    "X592.593_5nm",
    "X597.593_5nm",
    "X602.593_5nm",
    "X607.593_5nm",
    "X612.593_5nm",
    "X617.593_5nm",
    "X622.593_5nm",
    "X627.593_5nm",
    "X632.593_5nm",
    "X637.593_5nm",
    "X642.593_5nm",
    "X647.593_5nm",
    "X652.593_5nm",
    "X657.593_5nm",
    "X662.593_5nm",
    "X667.593_5nm",
    "X672.593_5nm",
    "X677.593_5nm",
    "X682.593_5nm",
    "X687.593_5nm",
    "X692.593_5nm",
    "X697.593_5nm",
    "X702.593_5nm",
    "X707.593_5nm",
    "X712.593_5nm",
    "X717.593_5nm",
    "X722.593_5nm",
    "X727.593_5nm",
    "X732.593_5nm",
    "X737.593_5nm",
    "X742.593_5nm",
    "X747.593_5nm",
    "X752.593_5nm",
    "X757.593_5nm",
    "X762.593_5nm",
    "X767.593_5nm",
    "X772.593_5nm",
    "X777.593_5nm",
    "X782.593_5nm",
    "X787.593_5nm",
    "X792.593_5nm",
    "X797.593_5nm",
    "X802.593_5nm",
    "X807.593_5nm",
    "X812.593_5nm",
    "X817.593_5nm",
    "X822.593_5nm",
    "X827.593_5nm",
    "X832.593_5nm",
    "X837.593_5nm",
    "X842.593_5nm",
    "X847.593_5nm",
    "X852.593_5nm",
    "X857.593_5nm",
    "X862.593_5nm",
    "X867.593_5nm",
    "X872.593_5nm",
    "X877.593_5nm",
    "X882.593_5nm",
    "X887.593_5nm",
    "X892.593_5nm",
    "X897.593_5nm",
    "X902.593_5nm",
    "X907.593_5nm",
    "X912.593_5nm",
    "X917.593_5nm",
    "X922.593_5nm",
    "X927.593_5nm",
    "X932.593_5nm",
    "X937.593_5nm",
    "X942.593_5nm",
    "X947.593_5nm",
    "X952.593_5nm",
    "X957.593_5nm",
    "X962.593_5nm",
    "X967.593_5nm",
    "X972.593_5nm",
    "X977.593_5nm",
    "X982.593_5nm",
    "X987.593_5nm",
    "X992.593_5nm"
)

ground_spectra <- veg_indices[,used_bands]

In [ ]:
head(ground_spectra)

In [ ]:
# Targets 
targets <- veg_indices[!is.na(veg_indices$Functional_group1),"Functional_group1"] %>% as.factor()
# weights
weights_by_pft <- targets_to_weights(targets)

In [ ]:
# image-based validation
uav_speclib_df <- read.csv(
    "Data/D_002_Image_SpecLib_Derivs.csv",
    header = TRUE
    )
print(colnames(uav_speclib_df))

image_validation <- uav_speclib_df[,7:(ncol(uav_speclib_df))]
validation_labels <- uav_speclib_df$Functional_group1 %>% as.factor()
#levels(validation_labels) <- c(levels(validation_labels), "Forb") 
print(colnames(image_validation))

In [ ]:
image_band_file <- "Data/Ground_Validation/PFT_Image_spectra/PFT_Image_SpectralLib_Clean.csv"
image_bands_base <- read.csv(image_band_file, header = TRUE)
head(image_bands_base)

In [ ]:
head(uav_speclib_df)

## Base transformation
This removes infinity, outliers and NAs from the data.  

In [ ]:
numeric_data <- veg_indices[!is.na(veg_indices$Functional_group1),35:(ncol(veg_indices)-1)]
numeric_data <- inf_to_na(numeric_data)
imputed_data_1 <- impute_spectra(numeric_data)
imputed_data_no_outliers <- outliers_to_na(imputed_data_1)
imputed_data <- impute_spectra(imputed_data_no_outliers)
outlier_indices <- detect_outliers_columnwise(imputed_data[,1:95])
filtered_data <- imputed_data[!outlier_indices,]
hist(dist(as.matrix(imputed_data)))
min_max_scaled_data <- columnwise_min_max_scale(imputed_data)

In [ ]:
print(colnames(imputed_data))

In [ ]:
print(colnames(image_bands_base))

## Transform the Image-based Data

In [ ]:
veg_index_names <- read.csv("assets/vegIndicesUsed.csv")$x
validation_indices <- get_vegetation_indices(image_bands_base[,7:ncol(image_bands_base)], NULL)
# drop NAs

validation_indices <- inf_to_na(validation_indices)
validation_indices <- impute_spectra(validation_indices)
validation_indices <- outliers_to_na(validation_indices)
validation_indices <- impute_spectra(validation_indices)


min_max_scaled_validation <- columnwise_min_max_scale(validation_indices)

#hist(as.matrix(min_max_scaled_validation))

In [ ]:
print(colnames(image_bands_base[,7:ncol(image_bands_base)]))

In [ ]:
validation_bands <- resample_df(image_bands_base[,7:ncol(image_bands_base)],normalize = FALSE,    min_wavelength = 397.593+5,
    max_wavelength = 995.716,
    delta = 5)

In [ ]:
head(validation_bands*100)
head(imputed_data)

In [ ]:
print(colnames(validation_bands))

In [ ]:
image_weights <- targets_to_weights(validation_labels %>% as.factor())

In [77]:
# generate the test data
set.seed(61718L)
print(validation_labels)
permutation <- permute::shuffle(length(validation_labels))
val_t_chars <- validation_labels[permutation] %>% as.character()
print(val_t_chars[1:5])
counts <- c(0,0,0,0,0,0,0,0,0)
samples <- vector(mode = "logical", length=length(validation_labels))

for(i in seq_along(val_t_chars)){
    samples[[i]] <- FALSE
    if(val_t_chars[[i]] == "Abiotic"){
        if(counts[[1]]<31){
            samples[[i]] <-  TRUE
            counts[[1]] <- counts[[1]] + 1
        }

    } else if (val_t_chars[[i]] == "Graminoid"){
        if(counts[[2]] < 31){
            samples[[i]] <-  TRUE
            counts[[2]] <- counts[[2]] + 1
        }
    } else if (val_t_chars[[i]] == "Forb"){
        if(counts[[3]] < 31){
            samples[[i]] <-  TRUE
            counts[[3]] <- counts[[3]] + 1
        }
    } else if (val_t_chars[[i]] == "Lichen"){
        if(counts[[4]] < 31){
            samples[[i]] <-  TRUE
            counts[[4]] <- counts[[4]] + 1
        }
    } else if (val_t_chars[[i]] == "Moss"){
        if(counts[[5]] < 31){
            samples[[i]] <-  TRUE
            counts[[5]] <- counts[[5]] + 1
        }
    } else if (val_t_chars[[i]] == "ShrubDecid"){
        if(counts[[6]] < 31){
            samples[[i]] <-  TRUE
            counts[[6]] <- counts[[6]] + 1
        }
    } else if (val_t_chars[[i]] == "ShrubEvergreen"){
        if(counts[[7]] < 31){
            samples[[i]] <-  TRUE
            counts[[7]] <- counts[[7]] + 1
        }
    } else if (val_t_chars[[i]] == "TreeConifer"){
        if(counts[[8]] < 31){
            samples[[i]] <-  TRUE
            counts[[8]] <- counts[[8]] + 1
        }
    } else if (val_t_chars[[i]] == "TreeBroadleaf"){
        if(counts[[9]] < 31){
            samples[[i]] <-  TRUE
            counts[[9]] <- counts[[9]] + 1
        }
    }
}


temp <- validation_labels[permutation] %>% as.factor()
test_labels <- temp[as.vector(samples)]
temp <- cbind(validation_bands[permutation,], 100*validation_bands[permutation,]) %>% as.data.frame()
test_data <- temp[as.vector(samples),]



   [1] TreeBroadleaf  TreeBroadleaf  TreeBroadleaf  TreeBroadleaf 
   [5] TreeBroadleaf  TreeBroadleaf  TreeBroadleaf  TreeBroadleaf 
   [9] TreeBroadleaf  TreeBroadleaf  TreeBroadleaf  TreeBroadleaf 
  [13] TreeBroadleaf  TreeBroadleaf  TreeBroadleaf  TreeBroadleaf 
  [17] TreeBroadleaf  TreeBroadleaf  TreeBroadleaf  TreeBroadleaf 
  [21] TreeBroadleaf  TreeBroadleaf  TreeBroadleaf  TreeBroadleaf 
  [25] TreeBroadleaf  TreeBroadleaf  TreeBroadleaf  TreeBroadleaf 
  [29] TreeBroadleaf  TreeBroadleaf  TreeBroadleaf  TreeBroadleaf 
  [33] TreeBroadleaf  TreeBroadleaf  TreeBroadleaf  TreeBroadleaf 
  [37] TreeBroadleaf  TreeBroadleaf  TreeBroadleaf  TreeBroadleaf 
  [41] TreeBroadleaf  TreeBroadleaf  TreeBroadleaf  TreeBroadleaf 
  [45] TreeBroadleaf  TreeBroadleaf  TreeBroadleaf  TreeBroadleaf 
  [49] TreeBroadleaf  TreeBroadleaf  TreeBroadleaf  TreeBroadleaf 
  [53] TreeBroadleaf  TreeBroadleaf  TreeBroadleaf  TreeBroadleaf 
  [57] TreeBroadleaf  TreeBroadleaf  TreeBroadleaf  TreeBroadl

In [ ]:
create_matched_data <- function(left_df, right_df, cols = c("targets", "targets")){
    
    shared_levels <- intersect(
        levels(as.factor(left_df[, cols[1]])), 
        levels(as.factor(right_df[, cols[[2]]]))
        )

    # store inermediate results in a list
    level_dfs_l <- NULL
    level_dfs_r <- NULL

    for( level in shared_levels){
        filtered_left <- left_df[(left_df[,cols[[1]]] == level),]
        filtered_right <- right_df[right_df[,cols[[2]]] == level,]

        permutation_left <- permute::shuffle(nrow(filtered_left))
        permutation_right <- permute::shuffle(nrow(filtered_right))

        num_records <- min(nrow(filtered_left), nrow(filtered_right))
        
        if(num_records > 0){
            if(is.null(level_dfs_l)){
                level_dfs_l <- filtered_left[1:num_records,]
            } else {
                level_dfs_l <- rbind(level_dfs_l, filtered_left[permutation_left[1:num_records],])
            }

            if(is.null(level_dfs_r)){
                level_dfs_r <- filtered_right[1:num_records,]
            } else {
                level_dfs_r <- rbind(level_dfs_r, filtered_right[permutation_right[1:num_records],])
            }

        }
    }

    return(list(
        left = level_dfs_l,
        right = level_dfs_r
    ))


}

In [78]:
# generate the test data
set.seed(61718L)
permutation <- permute::shuffle(targets)
val_t_chars <- targets[permutation] %>% as.character()
print(val_t_chars[1:5])
counts <- c(0,0,0,0,0,0,0,0,0)
samples <- vector(mode = "logical", length=length(targets))

for(i in seq_along(val_t_chars)){
    samples[[i]] <- FALSE
    if(val_t_chars[[i]] == "Abiotic"){
        if(counts[[1]]<31){
            samples[[i]] <-  TRUE
            counts[[1]] <- counts[[1]] + 1
        }

    } else if (val_t_chars[[i]] == "Graminoid"){
        if(counts[[2]] < 31){
            samples[[i]] <-  TRUE
            counts[[2]] <- counts[[2]] + 1
        }
    } else if (val_t_chars[[i]] == "Forb"){
        if(counts[[3]] < 31){
            samples[[i]] <-  TRUE
            counts[[3]] <- counts[[3]] + 1
        }
    } else if (val_t_chars[[i]] == "Lichen"){
        if(counts[[4]] < 31){
            samples[[i]] <-  TRUE
            counts[[4]] <- counts[[4]] + 1
        }
    } else if (val_t_chars[[i]] == "Moss"){
        if(counts[[5]] < 31){
            samples[[i]] <-  TRUE
            counts[[5]] <- counts[[5]] + 1
        }
    } else if (val_t_chars[[i]] == "ShrubDecid"){
        if(counts[[6]] < 31){
            samples[[i]] <-  TRUE
            counts[[6]] <- counts[[6]] + 1
        }
    } else if (val_t_chars[[i]] == "ShrubEvergreen"){
        if(counts[[7]] < 31){
            samples[[i]] <-  TRUE
            counts[[7]] <- counts[[7]] + 1
        }
    } else if (val_t_chars[[i]] == "TreeConifer"){
        if(counts[[8]] < 31){
            samples[[i]] <-  TRUE
            counts[[8]] <- counts[[8]] + 1
        }
    } else if (val_t_chars[[i]] == "TreeBroadleaf"){
        if(counts[[9]] < 31){
            samples[[i]] <-  TRUE
            counts[[9]] <- counts[[9]] + 1
        }
    }
}


temp <- targets[permutation] %>% as.factor()
train_labels <- temp[as.vector(samples)]
temp <- transformed_data[permutation,] %>% as.data.frame()
train_data <- temp[as.vector(samples),]


[1] "Lichen"     "Lichen"     "Abiotic"    "Graminoid"  "ShrubDecid"


In [79]:
# match the samples between the data and create a matched dataset
joined_train <- train_data
joined_train$targets <- train_labels

joined_test <- test_data
joined_test$targets <- test_labels




In [80]:
matched_data <- create_matched_data(
    joined_test %>% as.data.frame(),
    joined_train %>% as.data.frame()
)

In [81]:
build_columnwise_sensor_correction_model <- function(
    left_df, 
    right_df, 
    ignore_cols = NULL,
    verbose = TRUE
) {

    used_cols <- intersect(
        colnames(left_df),
        colnames(right_df)
    )

    if(!is.null(ignore_cols)){
        used_cols <- setdiff(used_cols, ignore_cols)
    }


    models  <- list()

    for(col in used_cols){
        left_vec <- sort(left_df[,col])
        right_vec <- sort(right_df[,col])
        if(is.numeric(left_vec) & is.numeric(right_vec)){
            model <- lm(
                left_vec~right_vec,
                )

            if(verbose){
                print(summary(model))
            }
            models[[col]] <- model
        }
    }

    return(models)
}

In [82]:
apply_sensor_correction_model <- function(
    models,
    data,
    ignore_cols = NULL
){
    used_cols <- colnames(data)
     if(!is.null(ignore_cols)){
        used_cols <- setdiff(used_cols, ignore_cols)
    }

    df_corrected <- as.data.frame(data)

    for(col in used_cols){
        if(!is.null(models[[col]])){
            print(paste0("Correcting ", col))
            model_intercept <- summary(models[[col]])$coefficients[1,1]
            model_slope <- summary(models[[col]])$coefficients[2,1]
            df_corrected[,col] <- model_intercept + (data[,col]*model_slope)
        }
    }

    return(df_corrected)
}

In [83]:
correction_models <- build_columnwise_sensor_correction_model(matched_data$left, matched_data$right)


Call:
lm(formula = left_vec ~ right_vec)

Residuals:
       Min         1Q     Median         3Q        Max 
-7.841e-04 -2.195e-04 -1.757e-05  9.145e-05  8.171e-04 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 0.000e+00  3.713e-05     0.0        1    
right_vec   1.000e+00  9.986e-03   100.1   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.0002706 on 222 degrees of freedom
Multiple R-squared:  0.9783,	Adjusted R-squared:  0.9782 
F-statistic: 1.003e+04 on 1 and 222 DF,  p-value: < 2.2e-16


Call:
lm(formula = left_vec ~ right_vec)

Residuals:
       Min         1Q     Median         3Q        Max 
-0.0013792 -0.0003947 -0.0001982  0.0004557  0.0009660 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -2.318e-18  6.642e-05    0.00        1    
right_vec    1.000e+00  1.673e-02   59.76   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ 

In [76]:
transformed_data <- apply_sensor_correction_model(
    correction_models,
    imputed_data
)

head(transformed_data)
head(imputed_data)

[1] "Correcting Boochs"
[1] "Correcting Boochs2"
[1] "Correcting CARI"
[1] "Correcting Carter"
[1] "Correcting Carter2"
[1] "Correcting Carter3"
[1] "Correcting Carter4"
[1] "Correcting Carter5"
[1] "Correcting Carter6"
[1] "Correcting CI"
[1] "Correcting CI2"
[1] "Correcting ClAInt"
[1] "Correcting CRI1"
[1] "Correcting CRI2"
[1] "Correcting CRI3"
[1] "Correcting CRI4"
[1] "Correcting D1"
[1] "Correcting D2"
[1] "Correcting Datt"
[1] "Correcting Datt2"
[1] "Correcting Datt3"
[1] "Correcting Datt4"
[1] "Correcting Datt5"
[1] "Correcting Datt6"
[1] "Correcting DD"
[1] "Correcting DDn"
[1] "Correcting DPI"
[1] "Correcting DWSI4"
[1] "Correcting EGFN"
[1] "Correcting EGFR"
[1] "Correcting EVI"
[1] "Correcting GDVI2"
[1] "Correcting GDVI3"
[1] "Correcting GDVI4"
[1] "Correcting GI"
[1] "Correcting Gitelson"
[1] "Correcting Gitelson2"
[1] "Correcting GMI1"
[1] "Correcting GMI2"
[1] "Correcting GreenNDVI"
[1] "Correcting Maccioni"
[1] "Correcting MCARI"
[1] "Correcting MCARIOSAVI"
[1] "Corre

,Boochs,Boochs2,CARI,Carter,Carter2,Carter3,Carter4,Carter5,Carter6,CI,⋯,X942.593_5nm,X947.593_5nm,X952.593_5nm,X957.593_5nm,X962.593_5nm,X967.593_5nm,X972.593_5nm,X977.593_5nm,X982.593_5nm,X987.593_5nm
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.0009942016,0.0008384083,0.2721208,3.522863,0.5453765,0.3881126,0.7290414,1.086916,0.08080786,0.9826569,⋯,18.98678,19.54539,18.85865,18.99266,18.66549,19.09837,19.00169,19.37496,18.78869,18.75065
2,0.0009772320,0.0008443052,0.2421938,3.053382,0.5228795,0.3896779,0.7124224,1.073493,0.07270964,0.9818436,⋯,17.10217,17.82967,16.95890,17.12457,16.70182,17.10762,16.88280,17.33468,16.39516,16.33555
3,0.0014131861,0.0009797791,0.3122896,3.684972,0.5482566,0.4138796,0.7408949,1.064955,0.10880536,0.9831065,⋯,20.92619,21.28434,20.76231,20.87325,20.65461,21.10260,21.09776,21.34353,21.03680,20.96758
4,0.0015138358,0.0010271853,0.3203071,3.701746,0.5402521,0.4109559,0.7310695,1.059485,0.11590111,0.9814928,⋯,21.78027,22.05772,21.60820,21.68805,21.49501,21.94504,21.97842,22.16190,21.95790,21.86710
5,0.0013410745,0.0009931145,0.2946876,3.709181,0.5395068,0.4063555,0.7330813,1.058068,0.09910808,0.9802976,⋯,19.93632,20.38765,19.77302,19.89083,19.60858,20.03817,19.97764,20.29484,19.85058,19.81078
6,0.0043001321,0.0054863379,0.2569642,2.518464,0.2550140,0.1882497,0.4437803,1.566580,0.06526272,0.9645370,⋯,26.80896,26.58764,26.56448,26.51761,26.55256,27.07691,27.42261,27.33417,27.92214,27.80292


,Boochs,Boochs2,CARI,Carter,Carter2,Carter3,Carter4,Carter5,Carter6,CI,⋯,X942.593_5nm,X947.593_5nm,X952.593_5nm,X957.593_5nm,X962.593_5nm,X967.593_5nm,X972.593_5nm,X977.593_5nm,X982.593_5nm,X987.593_5nm
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.1817917,0.1007417,38.29230,4.176483,0.7481783,0.6363165,0.8663692,1.117048,13.0818,1.0250352,⋯,26.46858,26.59656,26.71230,26.78679,26.84674,26.92454,27.04128,27.20405,27.38601,27.55660
2,0.1779167,0.1020167,28.80530,2.891436,0.7004412,0.6404789,0.8394989,1.101463,11.0986,1.0235607,⋯,21.29147,21.41355,21.52685,21.62140,21.70847,21.80113,21.90688,22.02912,22.16226,22.29784
3,0.2774667,0.1313083,51.02598,4.620200,0.7542897,0.7048361,0.8855345,1.091549,19.9382,1.0258505,⋯,31.79622,31.84974,31.90839,31.98678,32.05158,32.08263,32.12036,32.19709,32.29237,32.38387
4,0.3004500,0.1415583,53.56754,4.666115,0.7373047,0.6970615,0.8696484,1.085199,21.6759,1.0229248,⋯,34.14242,34.18603,34.21727,34.23974,34.25062,34.25074,34.25434,34.27278,34.30264,34.34254
5,0.2610000,0.1341917,45.44606,4.686466,0.7357233,0.6848281,0.8729010,1.083553,17.5634,1.0207577,⋯,29.07701,29.14092,29.20809,29.27031,29.31446,29.34322,29.40614,29.53721,29.70353,29.86498
6,0.9367000,1.1057000,33.48758,1.427278,0.1320466,0.1048420,0.4051465,1.673969,9.2749,0.9921833,⋯,47.95642,47.87045,47.74562,47.59385,47.48448,47.45815,47.44645,47.39159,47.31926,47.26754


## PCA 
This is where we calcuate PCA for the ground and image spectra

In [ ]:
# fit a PCA to the ground spectra
pca_fit <- stats::prcomp(imputed_data[,1:(ncol(numeric_data) - 66)], center = FALSE, scale. = FALSE)
print(summary(pca_fit))
pca_training_data <- predict(pca_fit, imputed_data[,1:(ncol(numeric_data) - 66)])[,1:64]
boxplot(vegan::scores(pca_training_data)[,2]~targets)

## Standardization
This cell standardizes the input to center at zero with standard deviation one.

In [ ]:
# standardization
indice_standardizer <- caret::preProcess(imputed_data[,1:95])
standardized_indices <- predict(indice_standardizer, imputed_data[,1:95])

val_standardizer <- caret::preProcess(validation_indices)
standardized_validation <- predict(val_standardizer, validation_indices)

In [ ]:
hist(standardized_indices$Carter, breaks = 20)
hist(standardized_validation$Carter, breaks = 20)

## Min-Max Scaling
This executes the min-man scalaing (to make the data on the scale [0,1])

In [ ]:
# plots
#hist(min_max_scaled_validation %>% as.matrix())
#hist(min_max_scaled_data %>% as.matrix())
print(colnames(validation_indices))
print(str(pca_fit))
#pca_validation_data <- predict(pca_fit, validation_indices)[,1:64] %>% as.data.frame()
#boxplot(vegan::scores(pca_validation_data)[,2]~validation_labels)

## KS Tests of Transferrability
These next few cells test whether the veg indices are similarly distributed (i.e. could be samples drawn from the same distribution)

The hypothesis is that columns (veg indices) that pass this test can safely be used across models and conditions (are transferrable)

In [ ]:
source("Functions/lecospectR.R")
ks_test_results <- test_transferrability(matched_data$left, matched_data$right)
print(ks_test_results)

## t-SNE
Examine the clusters in the data via *t*-SNE

In [ ]:
library(Rtsne)
unique_indices <- imputed_data[!duplicated(imputed_data),1:95]
normalized_veg_indices <- Rtsne::normalize_input(
    unique_indices %>% 
    as.matrix()
    )
embedding_2D <- Rtsne::Rtsne(normalized_veg_indices)
print(names(embedding_2D))

plot(embedding_2D$Y, col = as.factor(targets))
par(xpd=T)
legend("topright", legend = unique(targets), col = seq_along(unique(targets)),pch = 1)

## Vector Quantization Classifier
This fits a LVQ classifier to the data and then 

In [ ]:
print(length(validation_labels))
print(nrow(min_max_scaled_validation))

## Train-Test Split

Perform an 80-20 split on the data (use the split on the fly during the grid search)

In [ ]:
grd_train_idx <- caTools::sample.split(targets, SplitRatio = 0.8)

In [ ]:
img_train_idx <- caTools::sample.split(validation_labels, SplitRatio = 0.8)

## Random Forest
trains a random forest model

In [ ]:

rf_model <- ranger::ranger(
    num.trees = 1024,
    case.weights = weights_by_pft,
    importance = "impurity",
    classification = TRUE,
    x = transformed_data,
    y = targets
)

print(rf_model)

In [ ]:
predictions <- predict(rf_model, validation_indices)$predictions %>% 
    as.factor()
confusion_matrix <- caret::confusionMatrix(
    predictions, 
    validation_labels, 
    mode = "everything")
print(confusion_matrix)

In [ ]:
save(rf_model, file="mle/models/gs/rf_base_2.rda")

In [ ]:
important_variables <- sort(rf_model$variable.importance, decreasing = TRUE)
print(important_variables)

In [ ]:
important_variable_names<- names(important_variables)
print(important_variable_names)

In [ ]:
print(test_labels %>% table())

In [ ]:
head(matched_data$left)

In [ ]:
head(matched_data$right)

In [71]:
write_tukey_symmetric_difference_plots <- function(
    left_df,
    right_df,
    save_directory = "./", 
    ignore_cols = NULL
) {
    used_cols <- intersect(
        colnames(left_df),
        colnames(right_df)
    )

    if(!is.null(ignore_cols)){
        used_cols <- setdiff(used_cols, ignore_cols)
    }

    for(col in used_cols){
        y_l <- left_df[,col] 
        r_l <- right_df[,col]
        if(is.numeric(y_l) & is.numeric(r_l)){
            plt <- create_tukey_symmetric_difference_plot(
                y_l,r_l
            )
           ggsave(
            paste0(save_directory, col, ".png"),
            plot = plt
           ) 
        }



    }
}

In [72]:
create_tukey_symmetric_difference_plot <- function(
    left_vec,
    right_vec
) {
    average_vec <- (left_vec + right_vec) / 2
    difference_vec <- (left_vec - right_vec)

    min_average <- min(average_vec)
    max_average <- max(average_vec)

    sd_difference <- sd(difference_vec)
    mean_difference <- mean(difference_vec)
    upper_bound <- mean_difference + (1.96*sd_difference)
    lower_bound <- mean_difference - (1.96 * sd_difference)

    plt <- ggplot(
            data = NULL,
            aes(
                x=average_vec,
                y=difference_vec
            )
        ) +
        geom_point() + 
        geom_hline(
            yintercept=lower_bound,
            color="red"
        ) + 
        geom_hline(
            yintercept = upper_bound,
            color="red"
        ) +
        geom_hline(
            yintercept = mean_difference,
            color="blue"
        ) +
        ylab("Diff. Between Measures") +
        xlab("Average Measure")

    return(plt)
        
}

In [85]:
write_tukey_symmetric_difference_plots(
    matched_data$left,
    matched_data$right,
    save_directory = "figures/linearCorrectedPlots/"
)

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 

In [ ]:
write_tukey_symmetric_difference_plots(
    standardize_df(matched_data$left,ignore_cols=c("targets")),
    standardize_df(matched_data$right,ignore_cols=c("targets")),
    save_directory = "figures/standardizedDifferences/"
)

write_tukey_symmetric_difference_plots(
    columnwise_min_max_scale(matched_data$left,ignore_cols=c("targets")),
    columnwise_min_max_scale(matched_data$right,ignore_cols=c("targets")),
    save_directory = "figures/minMaxDifferences/"
)

write_tukey_symmetric_difference_plots(
    columnwise_robust_scale(matched_data$left,ignore_cols=c("targets")),
    columnwise_robust_scale(matched_data$right,ignore_cols=c("targets")),
    save_directory = "figures/scaledDifferences/"
)


# Grid Search

This next section defines all the essentials for the grid search across our different candidate models. 

## Candidates

### Models
* Random Forest
* Learned Vector Quantization (LVQ)
* k-Nearest Neighbor (kNN)

Could also consider Support Vector Machine (SVM), Gradient Boosted Trees (e.g. LightGBM, XGBoost), matched filtering, Logistic Regression, etc.

### Data/Transformations

For each of the image/training data sets, test the following:
* raw, 
* raw (no outliers)
* standardized (z-score standardization)
* standardized (z-score standardization, no outliers)
* min-max scaled
* min-max scaled (no outliers)
* PCA
* PCA no outliers

Need to also vary how many columns are included in the analysis

In [ ]:
print(dim(imputed_data))
print(dim(standardized_indices))
print(dim(min_max_scaled_data))
print(dim(validation_indices))
print(dim(standardized_validation))
print(dim(min_max_scaled_validation))


In [ ]:
print(colnames(imputed_data))

In [ ]:
print(colnames(validation_bands))

In [ ]:
# define the data sets to loop over
gs_train <- list(
    #pca_training_data,
    transformed_data,
    standardize_df(transformed_data),
    #standardized_indices[,important_variable_names],
    columnwise_min_max_scale(transformed_data),
    columnwise_robust_scale(transformed_data)
)

gs_preprocessing <- list(
    "None",
    "Standard Scaling (z-score)",
    "Min-Max Scaling",
    "Robust Scaling (Median/IQR)"
)

gs_test <- list(
    cbind(validation_indices, 100*validation_bands)[permutation,][samples,],
    cbind(standardized_validation, scale(validation_bands))[permutation,][samples,],
    cbind(min_max_scaled_validation, columnwise_min_max_scale(validation_bands))[permutation,][samples,],
    columnwise_robust_scale(
        cbind(validation_indices, validation_bands)[permutation,][samples,]
    )
    #pca_validation_data[permutation,][samples,],
#    min_max_scaled_validation[permutation,][samples,],
 #   standardized_validation[permutation,][samples,]
    #pca_validation_data[-img_train_idx]

)

gs_train_labels <- list(
    targets,
    targets,
    targets,
    targets,
    targets,
    targets
)

gs_samples <- list(
    test_labels,
    test_labels,
    test_labels,
    test_labels,
    test_labels,
    test_labels,
    test_labels
)

In [ ]:
gs_methods <- list(
    #"svmLinear",
    #"rmda",
    "RFlda",
    #"adaboost",
    #"xgbLinear",
    #"xgbTree",
    #"xgbDART",
    "svmRadialWeights",
    #"mda",
    "knn",
    #"lda",
    "ranger"
    #"hda"# heteroscedastic discriminant analysis
)
# add: PLS-LDA, kNN, SVM+poly Kernel, SVM+Exp Kernel, more boosting, 

In [ ]:
gs_weight_text <- c(
    "prior weights",
    NULL
)

gs_weights <- list(
    weights_by_pft,
    weights_by_pft,
    weights_by_pft,
    weights_by_pft,
    weights_by_pft,
    weights_by_pft,
    weights_by_pft,
    image_weights,
    image_weights,
    image_weights
)

fit_ctrl <- caret::trainControl(
    method = "repeatedcv",
    number = 10,
    repeats = 10,
    #classProbs = TRUE,
    allowParallel = TRUE
)

In [ ]:
getwd()

In [ ]:
for(i in seq_along(gs_train)){
    df <- data.frame(gs_train[[i]])
    test_df <- gs_test[[i]]
        
    # train and print intermediate results to console
    print("Beginning Training")
    model <- ranger::ranger(
        num.trees = 1000,
        case.weights = weights_by_pft,
        classification = TRUE,
        x=df,
        y=gs_train_labels[[i]]
    )
    print(model)

    model_predictions <- predict(
        model, 
        test_df
    )$prediction %>% as.factor()
    
    test_samples <- gs_samples[[i]] %>% as.factor()
    levels(test_samples) <- c(levels(test_samples), "Forb")

    confusion_matrix <- caret::confusionMatrix(
        model_predictions, 
        test_samples,
        mode = "everything"
    )

    model_id <- uuid::UUIDgenerate()

    # append performance data to the logs for later comparison
    sink(file = "mle/log_rf_3.txt", append = TRUE)
    print("-------------------------------------------------------")
    print("---------------------- Model Data ---------------------")
    
    print(paste0("Model Type: Ranger (Random Forest)"))
    print(paste0("Data Index: ",i))
    print(paste0("Model UUID: ", model_id))
    print("---------------------- Confusion Matrix ---------------------")
    print(confusion_matrix)
    print("---------------------- Class Distribution ---------------------")
    print(model_predictions %>% as.factor() %>% table())
    print("-------------------------------------------------------")
    sink(NULL)


    model_metadata <- list(
        uuid = model_id,
        variables = colnames(df),
        type = "Random Forest (Ranger)",
        preprocessing = gs_preprocessing[[i]],
        saved = paste0("mle/models/gs/", model_id, ".rda"),
        results = paste0("mle/experiments/gs/", model_id, "/")
    )

    metadata_save_path <- paste0("mle/metadata/", model_id, ".json")
    json_metadata_str <- rjson::toJSON(model_metadata)
    write(json_metadata_str, file=metadata_save_path)
    
    save(model, file = paste0("mle/models/gs/", model_id, ".rda"))


    
}

In [ ]:
R.Version()

In [ ]:
source("Functions/lecospectR.R")

In [ ]:
model_ids <- c(
    #"2303a2d3-f479-48b7-9840-c9f74b73836d",# ranger subset 10 >0.4
    #"6b0a0987-ff68-4f17-a3ca-7aa56284d726",# RFLDA >0.45, subset 11
    #"9f8fcf59-571d-4472-8750-7370692a2794",#RFLDA
    #"ab537b5-5141-4381-a464-5a0ff6cde137",#SVM
    #"58bee4bd-ea18-4fdb-823a-21463be3da41",#KNN predictions failed, may be clash with parallel pkg
    #"cf2b9f30-4956-4c0b-a1fe-79af7590cac9",#Ranger
    #"0c63dd43-79fe-45dd-b95f-ac8d6b2d7265",#Ranger, subset 1
    #"f7d408d7-5cd5-4214-b31c-ec8f75e197dc",# SVM, subset 3 <- issue, re-run
    #"e335d595-c095-4356-b967-d15c37479527",# ranger all
    #"b80328dd-9c40-4e99-8a9f-239121931fb4",# ranger subset 8
    #"f8ad4263-8470-43db-856b-4476dfc83ccc",# ranger subset 7
    #"1cd97e42-f7e1-4580-928b-1a42609a6549",# Ranger subset 5
    #"2b9d4a04-0586-4743-83ff-39e77a7345c7",#ranger subset 11
    #"6c3bedc5-11ae-423e-a858-13240650f14b",#RF-LDA substet 5
    #"f362ade9-43cd-4342-b62d-8f2f1d5c50b5",#RFLDA subset 9 <- this particular model returns all NAs???
    #"9c496675-d6b4-46d1-a3cc-4da9f69c702",# ranger subset 12, ~.4 <- wrong path
    #"rf_base",
    #"44e2506d-a587-4a27-acbb-1842f9fb421a",# SVM all
    #"02cdc9a4-2265-432b-8010-170048a94c01",# kNN, Subset 3
    #"7707394d-5755-4bf5-a8c9-e9f8d08e0aa8",# SVM, subset 4
    #"53bb89e1-22a9-4199-ac24-22232063dd79",# SVM subset 6
    #"cdb0d9c9-3f3e-4c3a-8277-336cc58fcaa8",# svm subset 7
    #"bef61c5e-9458-41df-9ce5-a30e45299e8f",# svm subset 10
    #"af392d97-067b-4391-8577-ead82f84fc08"#knn subset 12
    #"e335392b-73af-478d-a7f8-e3eaacea9774",#knn, subset 1 <- predictions failed, re-run
    ##"d41050ba-04b4-4507-a21e-d3743f59b793"
    #"5059eb96-fe79-43b7-bcc6-9dd54fbc3e83",
    #"654ae09e-666d-4eee-90a7-008b9b34b147",
    #"7920d1b5-4fc7-4ba0-b282-23fbecbc889a"

"31940dfa-a053-4398-b240-a33c768e5075"
)

#for(id in model_ids){
    id <- "rf_base"
    model <- load_model(
        paste0("mle/models/gs/", id, ".rda")
    )

 

    save_path <- paste0("mle/experiments/gs/", id, "/")
    #print(names(model))
print(model)
    if(!dir.exists(save_path)){
        dir.create(save_path)
    }
    #print(quadrats[[3]])


   results <- validate_model(
       model, 
       save_path, 
       normalize_input = FALSE,
       standardize_input = FALSE,
       scale_input = FALSE,
       robust_scale_input = FALSE, 
       cluster = NULL)
   aggregated_results <- aggregate_results(save_path)

    plot_by_pft(
        aggregated_results,
        save_path = paste0(save_path, "aggregate.html"),
        open = FALSE
    )
#
    write_validation_table(
        aggregated_results,
        save_path = paste0(save_path, "table.html"),
        open = FALSE
    )

    
#}

In [ ]:
   print(model$forest$independent.variable.names)

In [ ]:
library(kernlab)
sink(NULL)
print(colnames(standardized_indices[,1:95]))

In [ ]:
# time to build the SVMs directly
  # train and print intermediate results to console
        include_top <- 35
        df <- data.frame(standardized_indices[,1:95][,important_variable_names[1:include_top]])
        #print(colnames(df))
        #df$targets <-  as.factor(targets) 
        print("Beginning Training")
        model <- train(
            x=standardized_validation[permutation,][samples,][,important_variable_names[1:include_top]] %>% as.matrix,
            y=test_labels,
            method="knn",
            trControl = fit_ctrl
            #type = "C-bsvc",#"spoc-svc",
            #kernel = "rbfdot",
            #kpar = "automatic",
            #C = 60,
            #cross = 3
        )
        
        #print(model)

        model_predictions <- predict(
            model, 
            newdata=standardized_validation[permutation,][samples,][,important_variable_names[1:include_top]] %>% as.matrix
        )#$response %>% as.factor()
        #print(model_predictions)
        
        test_samples <- test_labels %>% as.factor()
        levels(test_samples) <- c(levels(test_samples), "Forb")

        confusion_matrix <- caret::confusionMatrix(
            model_predictions, 
            test_samples,
            mode = "everything"
        )

        model_id <- uuid::UUIDgenerate()

        # append performance data to the logs for later comparison
        sink(file = "mle/log_svm.txt", append = TRUE, type = "output")
        print("-------------------------------------------------------")
        print("---------------------- Model Data ---------------------")
        
        print(paste0("Model Type: SVM"))
        print(paste0("Data Index: ",i))
        print(paste0("Model UUID: ", model_id))
        print(paste0("Selected Variables: ", colnames(df)))
        print("---------------------- Confusion Matrix ---------------------")
        print(confusion_matrix)
        print("---------------------- Class Distribution ---------------------")
        print(model_predictions %>% as.factor() %>% table())
        print("-------------------------------------------------------")
        sink(NULL)

        
        save(model, file = paste0("mle/models/gs/", model_id, ".rda"))


In [ ]:
markdown_text <- "# Grid Search Results\n"
base_path <- "mle/experiments/gs/"

dirs <- list.dirs(base_path, full.names = FALSE, recursive = FALSE)
for(dir in dirs){
    files <- list.files(paste0(base_path, dir), pattern = ".html", full.names = TRUE)
    md_str <- paste0(
        dir,
        "\t [Graph](",
        files[1],
        ")\t|\t[Table](",
        files[2],
        ")\n"
    )
    markdown_text <- paste0(markdown_text, md_str)
    
}

write(markdown_text, file = "gs.md")

In [ ]:
make_tukey_mean_difference_plot(v1, v2, )

In [ ]:
source("Scripts/validation_defs.R")

bg_shape <- sf::st_read(shape_path_1)$CLASS_NAME

In [ ]:
bg_raster <- terra::rast("validation_saved_output.grd")

In [ ]:
rasterVis::levelplot(bg_raster)
#plot(bg_shape)

In [ ]:
write.csv(test_data, file="assets/test_data.csv")
write.csv(test_labels %>% as.data.frame(), file="assets/test_cl.csv")

In [ ]:
knn_LSModel <- LSModel(
    model = list(),
    predict_function <- function(df, model, ...){
        base_data <- read.csv("assets/test_data.csv")
        base_labels <- read.csv("assets/test_cl.csv")
        predictions <- class::knn(base_data, df, base_labels, k=5) %>% as.data.frame()
        colnames(predictions) <- c("z")
        return(predictions)
    }
)

# Results

These are the results of the grid search on the basics.  High performing models get to go through validation

## Top performers:
* Architectures: Random Forest, SVM, kNN, and RF-LDA.  A 5th would be nice, maybe PLS.
* variable selection can improve performance
* 